In [17]:
import torch
import numpy

from torch import tensor

from numpy import array

# Parameters

In [5]:
# Number of recognized words you put in input
nb_input = 3

# Number of classe, constant
nb_output = 6

# Number of hidden layers
nb_hidd_lay = 3

# RNN implementation

In [18]:
class RNN(nn.Module):
    def __init__(self, nb_inputs, nb_layers, nb_neurons, nb_outputs, learning_rate):
        super(RNN, self).__init__()
        self.layer = nn.Sequential(
            nn.RNN(nb_inputs, nb_layers, nb_neurons, nb_outputs, dropout=0.5),
            nn.Softmax(nb_outputs)
        )
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, inputs):
        x = self.layer(inputs)
        return x
    
    def learn(batch_list, num_epochs=1,):
        self.train()
        losses = []
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(num_epochs):
            for batch_idx, (data, target) in enumerate(batch_list):
                data, target = Variable(data.cuda()), Variable(target.cuda())
                
                output = self(data)
                loss = criterion(output, target)
                
                losses.append(loss.data.item())
                
                self.optimizer.zero_grad()
                
                loss.backward()
                self.optimizer.step()

                if batch_idx % 100 == 0 or batch_idx % 100 == 1 :
                    print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                            epoch, 
                            batch_idx * len(data), 
                            len(train_loader.dataset),
                            100. * batch_idx / len(train_loader), 
                            loss.data.item()), 
                            end='')